In [1]:
import pandas as pd
import numpy as np
import time
import re

In [2]:
# gp = pd.read_csv('../gameprograms230920.csv')
# gp['usage'] = '게임'

In [3]:
today = time.strftime('%y%m%d')
# pd.DataFrame(gp).to_csv(f'../gameprograms{today}.csv', index=None)

In [4]:
er_origin = pd.read_csv('../etc_requirements_utf.csv')

In [5]:
er_origin

,name,cpu,ram,gpu,dx,os,sto,spec_class
0,한컴오피스 2022,"2.4GHz 이상, 2코어",2GB,DirectX 9 이상,DirectX 9,Windows 7 SP1 이상,2GB,1
1,Microsoft 365,"1.6GHz 이상, 2코어",4GB,DirectX 9 이상,DirectX 9,"Windows 11, Windows 10, Windows 8.1, Windows S...",4GB,1
2,Office 2019,"1.6GHz 이상, 2코어",4GB,DirectX 9 이상,DirectX 9,"Windows OS: Windows 11, Windows 10, Windows 10...",4GB,1
3,Office 2021,"1.1GHz 이상, 2코어",4GB,DirectX 9 이상,DirectX 9,Windows 10 또는 Windows 11,4GB,1
4,4k 영상 시청,인텔 코어 i3-12세대 12100,8GB,NaN,NaN,NaN,0MB,1
...,...,...,...,...,...,...,...,...
100,Avid Sibelius,Any modern CPU,4GB,NaN,NaN,"Windows 10, 11",36GB,1
101,LMMS (Linux MultiMedia Studio),"1.5 GHz x86, x86_64, or ARM-based CPU with 2 c...",1GB,NaN,NaN,Windows 7,100MB,0
102,LMMS (Linux MultiMedia Studio),"2 GHz x86, x86_64, or ARM-based CPU with 4 cores",4GB,NaN,NaN,Windows 10,512MB,1
103,Cakewalk,Intel Core 2 Duo E8400,4GB,NVIDIA GeForce 510,NaN,Windows,200MB,0


In [6]:
er_origin.gpu

0            DirectX 9 이상
1            DirectX 9 이상
2            DirectX 9 이상
3            DirectX 9 이상
4                     NaN
              ...        
100                   NaN
101                   NaN
102                   NaN
103    NVIDIA GeForce 510
104    NVIDIA GeForce 510
Name: gpu, Length: 105, dtype: object

In [7]:
## GPU 시작.................
gfile = pd.read_csv('../part_gpu_wb230921.csv')
gbfile = pd.read_csv('../benchmark_gpu.csv')
gpu = gfile.name
gpu_bench = gfile.bench_mark
req_gpu = er_origin.gpu

In [8]:
gpu

0                  이엠텍 지포스 RTX 4070 Ti GAMINGPRO D6X 12GB
1              이엠텍 지포스 RTX 4060 Ti STORM X Dual OC D6 8GB
2       갤럭시 GALAX 지포스 RTX 4070 EX GAMER WHITE OC D6X 12GB
3                이엠텍 지포스 RTX 3060 STORM X Dual OC D6 12GB
4                    SAPPHIRE 라데온 RX 7600 PULSE OC D6 8GB
                              ...                        
1316                       액슬 지포스 GTX750 Ti 클래식 V2 D5 2GB
1317                          라데온 RX 560 14CU D5 2GB (중고)
1318                                     EVGA 파워링크 (해외구매)
1319                          FORSA 지포스 GT610 Plus D3 1GB
1320          매트록스 M9148 LP PCle x16 1GB DP/DVI Quad 앤아이피
Name: name, Length: 1321, dtype: object

In [10]:
req_gpu.loc[req_gpu.str.contains(r'\dGB', na=False)]

24                                      2GB of GPU VRAM
27    NVIDIA® GeForce RTX or GTX 960 series or highe...
29    Integrated or discrete GPU with at least 2GB o...
30    Integrated or discrete GPU with at least 2GB o...
33                                      2GB of GPU VRAM
37                                      RTX 3060 Ti 8GB
38                                         RTX 3070 8GB
39                                      RTX 3070 Ti 8GB
56                                      2GB of GPU VRAM
57                                      8GB of GPU VRAM
58                                      1GB of GPU VRAM
59                                      4GB of GPU VRAM
60                                      2GB of GPU VRAM
61                                      8GB of GPU VRAM
63                                      4GB of GPU VRAM
68                                      1GB of GPU VRAM
69                                      4GB of GPU VRAM
74                                      4GB of G

In [11]:
ben_reg.name.loc[ben_reg['name'].str.contains(r'\d RX', na=False)]

NameError: name 'ben_reg' is not defined

In [79]:
req_reg.loc[req_reg.str.contains(r'Rx \d', na=False)]

Series([], Name: gpu, dtype: object)

In [26]:
ignore_text = ['브릿지', 'Sync', '파워링크', 'Box', 'BOX', 'Hercules', '링크패널', 'G550', 'LP', '(']

In [12]:
gpu_regex_map = {'지포스': 'GeForce', '라데온': 'Radeon', '타이탄': 'TITAN',
                 '쿼드로': 'Quadro', 'DUAL': ' ', 'GT1': 'GT 1', 'GT6': 'GT 6',
                 'GT7': 'GT 7', 'GTX': 'GTX ', 'Pro': 'PRO', 'WX': 'WX ', 'W6800X': 'W6800',
                 'W6900X': 'RX 6900', r'RX(\d)': r'RX \1', '(\d)GT': '\1 GT'}
req_regex_map = {'(GEFORCE|(g|G)(E|e)?r?(F|f)orce).?': ' GeForce ', '(N|n)(v|V)(I|i)(D|d)(I|i)?(A|a).?': ' NVIDIA ',
                 '\xa0': '', '(RADEON|(R|r)adeon).?': ' Radeon ', ' TI | ti ': ' Ti ', ' Amd ': ' AMD ', '(g|G)(t|T)(X|x)': ' GTX ',
                 ' Gt ': ' GT ', ' INTEL |intel ': ' Intel ', ' hd |Hd ': ' HD ', r'(\d)HD ': r'\1 HD ', '(\d)(T|t)(I|i) ': r'\1 Ti',
                 r' R(x|X)(\d)': r'RX-\2', r'(\d)(g|G)(t|T)': r'\1 GT', r'GT(\d)': r'GT \1', '\[|\]': '', r'(\d)(G|g)b ?': r'\1GB',
                 'gts': 'GTS', '1024 ?MB': '1GB', '2048 ?MB': '2GB', '3072 ?MB': '3GB', '4098 ?MB': '4GB', r'HD(\d)': r'HD \1', r' r(\d) ': ' R\1 ',
                 r'GB([a-z])': 'GB \1', ',|\/|\||\(|\)': ' ', '\+': '', r'(\d)\sGB': r'\1GB', r'R(X|x) (\d)': r'RX-\2'}
bench_regex_map = {'Titan': 'TITAN', '\+': '', 'Quadro': 'Quadro ', 'Pro': 'PRO', 'RADEON': 'Radeon', '(A|a)ti ': 'ATI ',
                   ' +\/ +': ' ', r'RX (\d)': r'RX-\1'}
none_regex_map = {r'\s+': ' ', '\*': ''}

In [13]:
gpu_reg = gpu.replace(regex=gpu_regex_map).replace(regex=none_regex_map)
req_reg = req_gpu.replace(regex=req_regex_map).replace(regex=none_regex_map)
ben_reg = gbfile.replace(regex=bench_regex_map).replace(regex=none_regex_map)

In [14]:
g_names = gpu_reg.str.strip().str.split(r' +')
reqg_names = req_reg.str.strip().str.split(r' +')
bench_names = ben_reg.name.str.strip().str.split(r' +')

In [15]:
gbfile['count']

0       38968
1       34841
2       32146
3       31652
4       31173
        ...  
2550        1
2551        1
2552        1
2553        1
2554        1
Name: count, Length: 2555, dtype: int64

In [16]:
reqg_names

0            [DirectX, 9, 이상]
1            [DirectX, 9, 이상]
2            [DirectX, 9, 이상]
3            [DirectX, 9, 이상]
4                         NaN
                ...          
100                       NaN
101                       NaN
102                       NaN
103    [NVIDIA, GeForce, 510]
104    [NVIDIA, GeForce, 510]
Name: gpu, Length: 105, dtype: object

In [17]:
req_gpu.loc[3]

'DirectX 9 이상'

In [18]:
for lst in reqg_names:
    if type(lst) == float: continue
    if any(x == r' \d ' for x in lst):
        print(reqg_names.loc[reqg_names.str.contains(r' \d ', na=False)])

In [19]:
bench_names

0                           [GeForce, RTX, 4090]
1                           [GeForce, RTX, 4080]
2                           [Radeon, PRO, W7800]
3                       [GeForce, RTX, 4070, Ti]
4                         [Radeon, RX-7900, XTX]
                          ...                   
2550         [VIA, Chrome9, HC3, IGP, WDDM, 1.1]
2551    [Intel, 82852/82855, GM/GME, Controller]
2552             [RAGE, 128, PRO, AGP, 4X, TMDS]
2553                            [Quadro, 2, PRO]
2554                         [FireMV, 2200, PCI]
Name: name, Length: 2555, dtype: object

In [20]:
gpu_bench

0       31652
1       22560
2       26902
3       17144
4       16661
        ...  
1316     3901
1317     3641
1318        0
1319      304
1320       40
Name: bench_mark, Length: 1321, dtype: int64

In [21]:
reqg_names

0            [DirectX, 9, 이상]
1            [DirectX, 9, 이상]
2            [DirectX, 9, 이상]
3            [DirectX, 9, 이상]
4                         NaN
                ...          
100                       NaN
101                       NaN
102                       NaN
103    [NVIDIA, GeForce, 510]
104    [NVIDIA, GeForce, 510]
Name: gpu, Length: 105, dtype: object

In [22]:
g_names

0       [이엠텍, GeForce, RTX, 4070, Ti, GAMINGPRO, D6X, ...
1       [이엠텍, GeForce, RTX, 4060, Ti, STORM, X, Dual, ...
2       [갤럭시, GALAX, GeForce, RTX, 4070, EX, GAMER, WH...
3       [이엠텍, GeForce, RTX, 3060, STORM, X, Dual, OC, ...
4        [SAPPHIRE, Radeon, RX, 7600, PULSE, OC, D6, 8GB]
                              ...                        
1316        [액슬, GeForce, GTX, 750, Ti, 클래식, V2, D5, 2GB]
1317               [Radeon, RX, 560, 14CU, D5, 2GB, (중고)]
1318                                 [EVGA, 파워링크, (해외구매)]
1319             [FORSA, GeForce, GT, 610, Plus, D3, 1GB]
1320    [매트록스, M9148, LP, PCle, x16, 1GB, DP/DVI, Quad...
Name: name, Length: 1321, dtype: object

In [23]:
gfile

,name,price,link,company,product_seq,image,chipset_company,chipset,nm,base_clock,...,cooling_type,pan_number,length,thickness,pin,feature,as_years,bench_mark,reg_date,bookmark
0,이엠텍 지포스 RTX 4070 Ti GAMINGPRO D6X 12GB,1128990.0,https://prod.danawa.com/info/?pcode=18668606&c...,이엠텍,18668606,//img.danawa.com/prod_img/500000/606/668/img/1...,NVIDIA,RTX 4070 Ti,4.0,2310.0,...,11,3.0,328.9,63.40,100,7,3.0,31652,202301,NaN
1,이엠텍 지포스 RTX 4060 Ti STORM X Dual OC D6 8GB,565530.0,https://prod.danawa.com/info/?pcode=20274764&c...,이엠텍,20274764,//img.danawa.com/prod_img/500000/764/274/img/2...,NVIDIA,RTX 4060 Ti,4.0,2310.0,...,9,2.0,249.9,40.10,10000,7,3.0,22560,202305,NaN
2,갤럭시 GALAX 지포스 RTX 4070 EX GAMER WHITE OC D6X 12GB,844610.0,https://prod.danawa.com/info/?pcode=19773293&c...,갤럭시,19773293,//img.danawa.com/prod_img/500000/293/773/img/1...,NVIDIA,RTX 4070,NaN,1920.0,...,11,3.0,323.0,48.00,10000,5,3.0,26902,202304,NaN
3,이엠텍 지포스 RTX 3060 STORM X Dual OC D6 12GB,383300.0,https://prod.danawa.com/info/?pcode=13463144&c...,이엠텍,13463144,//img.danawa.com/prod_img/500000/144/463/img/1...,NVIDIA,RTX 3060,8.0,NaN,...,11,2.0,245.0,40.00,10000,7,3.0,17144,202102,NaN
4,SAPPHIRE 라데온 RX 7600 PULSE OC D6 8GB,376190.0,https://prod.danawa.com/info/?pcode=20315144&c...,SAPPHIRE,20315144,//img.danawa.com/prod_img/500000/144/315/img/2...,AMD(ATi),RX 7600,NaN,NaN,...,11,2.0,240.0,44.07,10000,1,3.0,16661,202305,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1316,액슬 지포스 GTX750 Ti 클래식 V2 D5 2GB,143000.0,https://prod.danawa.com/info/?pcode=7424305&ca...,액슬,7424305,//img.danawa.com/prod_img/500000/305/424/img/7...,NVIDIA,GTX750 Ti,28.0,1020.0,...,9,1.0,162.0,35.00,0,0,NaN,3901,201903,NaN
1317,라데온 RX 560 14CU D5 2GB (중고),NaN,https://prod.danawa.com/info/?pcode=6643450&ca...,NaN,6643450,//img.danawa.com/prod_img/500000/450/643/img/6...,AMD(ATi),해당없음,NaN,NaN,...,0,NaN,NaN,NaN,0,0,NaN,3641,201811,NaN
1318,EVGA 파워링크 (해외구매),22700.0,https://prod.danawa.com/info/?pcode=5406903&ca...,EVGA,5406903,//img.danawa.com/prod_img/500000/903/406/img/5...,NaN,해당없음,NaN,NaN,...,0,NaN,NaN,NaN,0,0,NaN,0,201708,NaN
1319,FORSA 지포스 GT610 Plus D3 1GB,48030.0,https://prod.danawa.com/info/?pcode=3928707&ca...,FORSA,3928707,//img.danawa.com/prod_img/500000/707/928/img/3...,NVIDIA,GT610,40.0,810.0,...,9,1.0,150.0,NaN,0,0,NaN,304,201603,NaN


In [27]:
min_bench = np.inf
lowest_idx = 0
for i in range(len(gfile)):
    if any([x in gfile.loc[i, 'name'] for x in ignore_text]): continue
    if gfile.loc[i, 'bench_mark'] <= min_bench:
        print(i, gfile.loc[i, 'bench_mark'], min_bench)
        min_bench = gfile.loc[i, 'bench_mark']
        lowest_idx = i
print(i, gfile.loc[lowest_idx, 'name'], gfile.loc[lowest_idx, 'bench_mark'])

0 31652 inf
1 22560 31652
3 17144 22560
4 16661 17144
6 12793 16661
64 12793 12793
66 2489 12793
67 2489 2489
115 632 2489
242 632 632
481 632 632
814 632 632
815 632 632
979 98 632
1229 98 98
1320 GIGABYTE 지포스 G210 D3 1GB 98


In [29]:
start = time.time()
bench_value_list = [None] * len(reqg_names)
for i in range(len(reqg_names)):
    rn = reqg_names.loc[i]
    if type(rn) == float:
        continue
    # if i == 236 or i == 237:
    #     print(f'\n\n{rn}')
    max_val = 0
    min_bench = np.inf
    bench_list = []
    for j in range(len(bench_names)):
        bn = bench_names.loc[j]
        res = len(set(rn) & set(bn))
        val = res / float(len(set(rn) | set(bn))) * 100
        bench = gbfile.loc[j, 'count']
        if val >= max_val and min_bench > bench:
            max_val = val
            min_bench = bench
            if val > max_val:
                bench_list = [bench]
                
            # if i == 236 or i == 237:
            #     print(j, val, bench, bench_names.loc[j])
    min_idx = 0
    if max_val == 0:
        continue
    for k in range(len(gpu_bench)):
        if gpu_bench.loc[k] >= min_bench and gpu_bench.loc[min_idx] > gpu_bench.loc[k]:
            min_idx = k
            # if i == 236 or i == 237: print(f'K: {k}, {gpu_bench.loc[k]}, {gpu.loc[k]}')
    if gpu_bench.loc == 0:
        continue;
    else:
        bench_value_list[i] = min_idx + 1
    # if i == 237: break
    percent = i / len(reqg_names) * 100
    print(f'\rgpu reqs: {i+1}/{len(reqg_names)} ({percent:.0f}%)', end='')
    
print('\nDone!')
end = time.time()
print(f'\n걸린 시간: {end-start:.2f} sec')

gpu reqs: 105/105 (99%)
Done!

걸린 시간: 4.07 sec


In [30]:
id_list = [None] * len(reqg_names)
for i in range(len(reqg_names)):
    rn = reqg_names.loc[i]
    if type(rn) == float:
        continue
    max_val = 0
    min_bench = np.inf
    for j in range(len(g_names)):
        gn = g_names.loc[j]
        res = len(set(rn) & set(gn))
        val = res / float(len(set(rn) | set(gn))) * 100
        if val >= max_val and min_bench >= gpu_bench.loc[j]:
            max_val = val
            min_bench = gpu_bench.loc[j]
            id_list[i] = f'{j+1}'
    percent = i / len(reqg_names) * 100
    print(f'\rgpu: {i+1}/{len(reqg_names)} ({percent:.0f}%)', end='')

gpu: 105/105 (99%)

In [169]:
## 1. 게임 사양 <-> 벤치 이름 매칭
## 2. 벤치 값 이상이면서 가장 작은 부품 매칭

In [31]:
gr = er_origin

In [32]:
# gr['cpu'] = bench_value_list

In [44]:
# gr 21 75 76
k = '26 42 59 63 69 74 78 80 84'.split(' ')

In [46]:
for k_ in k:
    bench_value_list[int(k_)] = 400

In [49]:
bench_value_list[93] = 1153

In [50]:
for i in range(len(reqg_names)):
    if type(req_gpu.loc[i]) == float or bench_value_list[i] == None: continue
    idx = int(bench_value_list[i])-1
    idx_og = int(id_list[i])-1
    print(f':{i:5d}:{req_gpu.loc[i]:35s}:\n:{gpu_bench.loc[idx_og]:5d}::{gpu.loc[idx_og]}:\n:{gpu_bench.loc[idx]:5d}::{" ".join(filter(None, g_names.loc[idx]))}:\n')

:    0:DirectX 9 이상                       :
:    0::EVGA 파워링크 (해외구매):
: 1599::NVIDIA Quadro P400 D5 2GB (해외구매):

:    1:DirectX 9 이상                       :
:    0::EVGA 파워링크 (해외구매):
: 1599::NVIDIA Quadro P400 D5 2GB (해외구매):

:    2:DirectX 9 이상                       :
:    0::EVGA 파워링크 (해외구매):
: 1599::NVIDIA Quadro P400 D5 2GB (해외구매):

:    3:DirectX 9 이상                       :
:    0::EVGA 파워링크 (해외구매):
: 1599::NVIDIA Quadro P400 D5 2GB (해외구매):

:   21:nVidia or ATI with support of OpenGL 1.5 or higher:
:    0::NVIDIA NV링크 브릿지 2-Slot:
:  612::GeForce GT 720 (중고):

:   22:Intel HD Graphics 5000 or later; NVIDIA GeForce GTX 700 or later; AMD Radeon R5 or later:
: 1752::지포스 GTX650 D5 1GB (중고):
:  612::GeForce GT 720 (중고):

:   23:Intel HD Graphics 5000 or later; NVIDIA GeForce GTX 700 or later; AMD Radeon R5 or later:
: 1752::지포스 GTX650 D5 1GB (중고):
:  612::GeForce GT 720 (중고):

:   24:2GB of GPU VRAM                    :
:  304::갤럭시 지포스 GT610 D3 2GB LP:
: 1599::NVIDIA Quadro P400 D5 2G

In [51]:
gr = er_origin
gr['gpu'] = bench_value_list

In [52]:
gr

,name,cpu,ram,gpu,dx,os,sto,spec_class
0,한컴오피스 2022,"2.4GHz 이상, 2코어",2GB,1153.0,DirectX 9,Windows 7 SP1 이상,2GB,1
1,Microsoft 365,"1.6GHz 이상, 2코어",4GB,1153.0,DirectX 9,"Windows 11, Windows 10, Windows 8.1, Windows S...",4GB,1
2,Office 2019,"1.6GHz 이상, 2코어",4GB,1153.0,DirectX 9,"Windows OS: Windows 11, Windows 10, Windows 10...",4GB,1
3,Office 2021,"1.1GHz 이상, 2코어",4GB,1153.0,DirectX 9,Windows 10 또는 Windows 11,4GB,1
4,4k 영상 시청,인텔 코어 i3-12세대 12100,8GB,NaN,NaN,NaN,0MB,1
...,...,...,...,...,...,...,...,...
100,Avid Sibelius,Any modern CPU,4GB,NaN,NaN,"Windows 10, 11",36GB,1
101,LMMS (Linux MultiMedia Studio),"1.5 GHz x86, x86_64, or ARM-based CPU with 2 c...",1GB,NaN,NaN,Windows 7,100MB,0
102,LMMS (Linux MultiMedia Studio),"2 GHz x86, x86_64, or ARM-based CPU with 4 cores",4GB,NaN,NaN,Windows 10,512MB,1
103,Cakewalk,Intel Core 2 Duo E8400,4GB,200.0,NaN,Windows,200MB,0


In [53]:
pd.DataFrame(gr).to_csv(f'../etcrequirements{today}.csv', index=None)